#### this notebook is for using scipy curve\_fit to tweak parameters in a known form - specifically for variations in $A_s$

In [1]:
import matplotlib.pyplot as plt
import numpy as np

from scipy.optimize import curve_fit

In [2]:
# Load Data
pars     = np.load('../CL_data/parameter_as3.npy')  # [H0, ombh2, omch2 ] x 100
lensed   = np.load('../CL_data/lensed_as3.npy')     # [C_2, ..., C_5000] x 100 (lensed)
unlensed = np.load('../CL_data/unlensed_as3.npy')     # [C_2, ..., C_5000] x 100 (unlensed)

In [3]:
np.min(pars[:,1])

1.2002724903798974

In [4]:
cosmos = pars[:,1]
ells = np.array([l for l in range(2, 5000)])

In [5]:
lensing = lensed/unlensed #lensing tail
camb_data_format = []
for i in range(len(lensing)):
    for j in range(len(lensing[i])):
        camb_data_format.append(lensing[i][j])

In [6]:
def func(X, b1, b2):
    #expected form of the lensing tail function for curve fitting
    ell = X[0]
    as_mag = X[1] / 3.043 - 1
    beta1 = 0.6838 * (1 + 0.00295281 * as_mag + 0.07715963 * as_mag**2)
    beta2 = 2803.1562 * (1 +0.01287662  * as_mag + -0.12985897 * as_mag**2)
    beta3 =  3752.098 * (1 + -0.10173588 *as_mag + 0.19153116  * as_mag**2)
    beta4 = 407.009 * (1 +  0.53069688 * as_mag + 0.14825538 * as_mag**2)
    alpha = 2.5867 * ( 1 + 1.40881037  * as_mag + 0.7482106  * as_mag**2)
    sigma = (1 + np.exp(-(ell - beta3)/beta4))**-1
    poly = (beta1 * (ell/beta2)**alpha -1)
    
    return (poly*sigma + 1).flatten() 

In [7]:
n = len(cosmos) #number of cosmologies
params = np.zeros((len(ells) * n, 2))   #ind variables
for i in range(n):
    params[(i)*len(ells):(i+1)*len(ells),0] = ells
for i in range(n):
    params[(i)*len(ells):(i+1)*len(ells),1] = cosmos[i]  #ommh2

In [8]:
answers, _ = curve_fit(func,(params[:,0], params[:, 1]),camb_data_format)  #doing the curve fitting

/tmp/ipykernel_19474/3226740531.py:1: OptimizeWarning: Covariance of the parameters could not be estimated
  answers, _ = curve_fit(func,(params[:,0], params[:, 1]),camb_data_format)  #doing the curve fitting


In [9]:
answers

array([1., 1.])